In [48]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import OrderedDict

class NetworkModel(nn.Module):

    def __init__(self, inputDim):

        # Initialize the network layers.

        super(NetworkModel, self).__init__()

        self.lin1 = nn.Linear(inputDim, inputDim, bias=False)
        self.lin2 = nn.Linear(inputDim, inputDim, bias=False)

    def forward(self, x):

        # A forward function
        # Linear function without activation

        x = self.lin1(x)
        x = self.lin2(x)
        
#         F -> temp -> WalshSpec
#         temp = F * W1'
#         WalshSpec = temp * W2' 
#             = F * W1' * W2'

        return x

class TrainModel():

    def __init__(self, model, device, learningRate, inputDim, batchSize, numberOfSteps, epochs):

        self.device = device
        self.net = model.to(self.device)
        self.optimizer = optim.Adam(self.net.parameters(), lr=learningRate)
        self.inputDim = inputDim
        self.batchSize = batchSize
        self.numberOfSteps = numberOfSteps
        self.epochs = epochs
        self.H = {}
        self.H[4] = torch.tensor([[1.0, 1.0, 1.0, 1.0], [1.0, -1.0, 1.0, -1.0], [1.0, 1.0, -1.0, -1.0], [1.0, -1.0, -1.0, 1.0]])
        self.H[8] = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                            [1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0],
                            [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0],
                            [1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0],
                            [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0],
                            [1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0],
                            [1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0],
                            [1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0]])
        self.perm = torch.randperm(pow(2, self.inputDim))
        print("Permutation of input numbers i.e. truth tables: \n", self.perm)
        self.perm = [[0.0]*(self.inputDim-len(bin(num)[2:]))+[float(i) for i in bin(num)[2:]] for num in self.perm]
        print("Binary version of permutation: \n", self.perm)

    def train(self,):
        
        for ep in range(self.epochs):
            
            for step in range(self.numberOfSteps):

                self.optimizer.zero_grad()
                input = torch.tensor(self.perm[step])
                output = self.net(input)
                loss = F.mse_loss(torch.matmul(input, self.H[self.inputDim]), output)
    #             print(input, torch.matmul(input, H[self.inputDim]), output)
                loss.backward()
                self.optimizer.step()

            weight_dict = OrderedDict(self.net.named_parameters())
            weightFunction = weight_dict['lin1.weight'].T @ weight_dict['lin2.weight'].T
            print("ep= ", ep, "\n", weightFunction, 6)
        
        weight_dict = OrderedDict(self.net.named_parameters())
        weightFunction = weight_dict['lin1.weight'].T @ weight_dict['lin2.weight'].T
        print("Final rounded weight function: \n", torch.round(weightFunction))



    def test(self,n):

#         test_samples = torch.tensor([[float(el) for el in row] for row in torch.randint(0, 2, (n, self.inputDim)).to(self.device)])
        test_samples = torch.tensor(self.perm[self.numberOfSteps:self.numberOfSteps+n])    
        preds = self.net(test_samples)
        outputs = torch.matmul(test_samples, self.H[self.inputDim])
        print("Test:")
        for i in range(len(preds)):
            print(outputs[i], "\n", torch.round(preds[i]), "\n\n")


def main():

    # Please change the inputs here
    inputDim = 8

    # Additional inputs may not be required to change.
    learningRate = 0.01
    batchSize = 1
    numberOfSteps = 50
    epochs = 20

    # Additional Initializer
    device = torch.device('cpu')
    model =  NetworkModel(inputDim)
    trainer = TrainModel(model, device, learningRate, inputDim, batchSize, numberOfSteps, epochs)
    trainer.train()
    trainer.test(10)

if __name__ == '__main__':
    main()

Permutation of input numbers i.e. truth tables: 
 tensor([ 16,  35, 154,  14, 151, 114,  44,  22, 174, 163,  25,  38, 141, 207,
        139, 133,  48,  17, 119, 109, 108,  73, 175,  13,   3,  64, 210, 111,
        245, 204,  47,  97,  80, 104, 103,  59,  29, 202,  78, 181, 194,  39,
          6, 209, 117, 200, 226,  15, 125, 250,  71, 233,   4, 172,  92, 254,
         24, 167, 252, 227,  83,  82, 186, 105, 196, 102,  27,  52, 182, 230,
        120, 149, 253,  90,  88, 247, 240, 123, 248, 169, 246, 147, 164, 220,
        122,   9, 170, 101,  31, 191,   1,  54, 242, 127, 110, 173,  98, 217,
        143, 219,  95, 165,  18,   5, 100, 144, 145,  21, 212, 234,  96, 159,
         62, 135,   7,  66, 215,  74, 129,  89, 238,  41,  55,  40,  63, 201,
        211, 197, 205, 185,  57, 228, 192, 157, 229,  76, 178, 249, 136, 112,
        134, 224, 222, 221, 124,  68,  30,  84, 225, 128, 216, 255,  32, 150,
          0, 138,  49, 251,  93,  53,  91,  36,  87, 214, 160,  33, 126, 199,
        166,  

ep=  2 
 tensor([[ 0.9826,  0.3916,  0.1983,  0.9183,  0.4126, -0.2266,  0.7104,  0.5025],
        [ 1.0529, -0.4001,  0.9671, -0.4911,  0.7325, -1.0108,  0.2156, -0.5228],
        [ 1.0883, -0.1010, -0.5090, -0.3681, -0.2811,  0.4225, -0.4449, -0.5169],
        [ 0.7439, -0.4108, -0.4234,  0.9467,  1.1508, -0.3916,  0.0740,  0.1272],
        [ 0.9452,  0.0488,  0.4103, -0.1128,  0.0304, -0.3080,  0.0459, -0.2836],
        [ 1.1426, -0.3265,  0.1093, -0.8377, -1.0666,  0.7178, -0.2788, -0.3616],
        [ 0.9099,  0.9987, -0.1960, -0.5823, -0.5601, -0.3576,  0.4080,  0.6300],
        [ 0.9666, -0.5424, -0.4959,  0.3660, -0.3717,  1.0287, -0.2887, -0.2276]],
       grad_fn=<MmBackward0>) 6
ep=  3 
 tensor([[ 1.0229,  0.4903,  0.4345,  1.1956,  0.2284, -0.0226,  1.0104,  0.7570],
        [ 1.0583, -0.4924,  1.1220, -0.7175,  0.9388, -1.0973,  0.1576, -0.7722],
        [ 1.0577, -0.0408, -0.8637, -0.6095, -0.0235,  0.2713, -0.6596, -0.7276],
        [ 0.8679, -0.6331, -0.8419,  0.9431,  1

ep=  15 
 tensor([[ 0.9998,  0.9997,  0.9999,  0.9999,  0.9998,  1.0001,  1.0021,  0.9991],
        [ 1.0001, -0.9998,  1.0001, -1.0000,  1.0003, -0.9998,  0.9989, -0.9995],
        [ 0.9997,  0.9997, -1.0001, -1.0001,  0.9999,  1.0003, -0.9979, -1.0010],
        [ 1.0001, -0.9999, -1.0000,  1.0001,  0.9999, -1.0002, -1.0008,  1.0004],
        [ 0.9999,  0.9999,  1.0000,  1.0000, -1.0001, -1.0000, -0.9994, -1.0003],
        [ 1.0000, -0.9999,  1.0000, -1.0000, -0.9999,  1.0002, -1.0001,  1.0001],
        [ 1.0002,  1.0002, -0.9999, -0.9999, -1.0000, -1.0002,  0.9984,  1.0007],
        [ 1.0000, -1.0000, -0.9999,  0.9999, -0.9999,  1.0002,  0.9999, -1.0000]],
       grad_fn=<MmBackward0>) 6
ep=  16 
 tensor([[ 1.0000,  0.9999,  1.0000,  1.0000,  1.0000,  1.0001,  1.0005,  0.9998],
        [ 1.0000, -1.0000,  1.0000, -1.0000,  1.0001, -1.0000,  0.9998, -0.9999],
        [ 0.9999,  0.9999, -1.0000, -1.0000,  1.0000,  1.0001, -0.9995, -1.0002],
        [ 1.0000, -1.0000, -1.0000,  1.0000, 

In [39]:
print([[0]*(4-len(bin(num)[2:]))+[int(i) for i in bin(num)[2:]] for num in [2, 4, 15, 6]])

[[0, 0, 1, 0], [0, 1, 0, 0], [1, 1, 1, 1], [0, 1, 1, 0]]
